### Basic Workflow

1. Load and explore the data
2. Preprocess the data
3. Extract features
4. Train the model
5. Evaluate the model
6. Analyze model behavior

In [ ]:
import os
import spacy

import numpy as np
import pandas as pd
import pickle
from typing import TypeAlias

In [ ]:
FOLDER = "/Users/johnhuang/Desktop/coding/Bag_of_Words/Data"
FILEPATH = f"{FOLDER}/Tweets_5K.csv"

In [ ]:
def load_data(filepath: str) -> tuple[list[str], list[int]]:
    """
    Loads Twitter data into two lists.

    Returns
    -------
    raw_tweets : list[str]
        A list of all Tweets in the dataset
    labels : list[int]
        A list of the sentiments corresponding to each raw tweet encoded as integers,
        -1 meaning negative, 0 meaning neutral, and 1 meaning positive
    """
    dataset = pd.read_csv(filepath)
    raw_tweets = dataset["text"].astype(str).tolist()
    labels=[]
    for label in dataset["sentiment"].astype(str).tolist():
      if label == "neutral":
        labels.append(0)
      elif label == "negative":
        labels.append(-1)
      else:
        labels.append(1)
    return (raw_tweets, labels)

In [ ]:
raw_tweets, labels = load_data(FILEPATH)
for p, label in zip(raw_tweets[:10], labels[:10]):
    print(f"{label}:\t{p}\n")

###Plot Distribution

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
pd.Series(labels).value_counts().plot.bar(title="Sentiment Distribution in Tweets")
plt.xticks(rotation=0)
plt.xlabel("Sentiment")
plt.ylabel("Number of Tweets")
plt.show()

###Initial Preprocess Data

In [ ]:
def preprocess(raw_X: list[str]) -> list[list[str]]:
    """
    Performs splitting on whitespace on all raw strings in a list.

    Parameters
    ----------
    raw_X : list[str]
        A list of raw strings (tweets)

    Returns
    -------
    list[list[str]]
        A list of preprocessed tweets (which are now lists of words)
    """
    preprocessed_tweets = []
    for string in raw_X:
      preprocessed_tweets.append(string.split())
    return preprocessed_tweets

### Define Featurization and Train and Test functions

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

class BOW_Classifier:
    """
    Attributes
    ----------
    clf : LogisticRegression
        A logistic regression classifier
    dv : DictVectorizer
        A dictionary vectorizer for turning dictionaries into matrices
    """

    def __init__(self):
        self.clf = LogisticRegression(max_iter=150)
        self.dv = DictVectorizer()

    def featurize(self, preproc_X: np.ndarray[list[str]], is_test: bool = False) -> csr_matrix:
        """
        Turns a list of preprocessed tweets into a binary bag-of-words matrix.

        Parameters
        ----------
        preproc_X : np.ndarray[list[str]]
            A list of preprocessed tweets
        is_test: bool, default=False
            Whether featurization should be done using features learned during training (is_test=True)
            or whether it should be done with features extracted from scratch using preproc_X (is_test=False)

        Returns
        -------
        csr_matrix
            A matrix with rows corresponding to tweets and columns corresponding to words
        """
        vocab = []
        for string in preproc_X:
          tweet_dict = {}
          for word in string:
            tweet_dict[word] = 1
          vocab.append(tweet_dict)
        if is_test:
          matrix = self.dv.transform(vocab)
        else:
          matrix = self.dv.fit_transform(vocab)
        return matrix

    def train(self, X_train: np.ndarray[list[str]], y_train: np.ndarray[int]):
        """
        Trains the BOW classifier on the given training data.

        Parameters
        ----------
        X_train : np.ndarray[list[str]]
            Preprocessed tweets for training
        y_train : np.ndarray[int]
            Sentiments corresponding to the tweets in X_train
        """
        # TODO
        BoW_matrix = self.featurize(X_train, is_test = False)
        self.clf.fit(BoW_matrix, y_train)

    def test(self, X_test: np.ndarray[list[str]]) -> np.ndarray[int]:
        """
        Classifies the given test data and returns predicted sentiments.

        Parameters
        ----------
        X_test : np.ndarray[list[str]]
            Preprocessed tweets for testing

        Returns
        -------
        y_pred : np.ndarray[int]
            Predicted sentiments for the tweets in X_test
        """
        # TODO
        BoW_test_matrix = self.featurize(X_test, is_test = True)
        return self.clf.predict(BoW_test_matrix)